In [1]:
import torch
import torchvision
from scipy.io import loadmat
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import os
from torchvision import transforms as T
from utils import datsprwtrans as prw
from torch import device
from torch.optim.lr_scheduler import StepLR
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2

## Transfer Learning Model

Function to generate our model.

In [2]:
def get_model(num_classes):
    """Create a object detection model."""
    
    # load a model pre-trained on MSCOCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

Lets define our custom dataset which will load in our images and annotations.

In [3]:
class PRWTransfer(torch.utils.data.Dataset):
    """Creates a PRW dataset for a Object Detection model."""
    
    def __init__(self, img_root, ann_root, transforms=None):
        """Instantiates the dataset."""
        
        # set the root directories for data.
        self.root_img = img_root
        self.root_ann = ann_root
        self.transforms = transforms
        
        # load all image files, sorting them to
        # ensure that they are aligned.
        self.imgs = list(sorted(os.listdir(img_root)))
        self.anns = list(sorted(os.listdir(ann_root)))

    def __getitem__(self, idx):
        """Gets a sample from the dataset."""
        
        # load images and bounding boxes.
        img_path = os.path.join(self.root_img, self.imgs[idx])
        ann_path = os.path.join(self.root_ann, self.anns[idx])
        ann = loadmat(ann_path)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        # get the height and width of the image
        image_width = img.shape[1]
        image_height = img.shape[0]
        img = cv2.resize(img, (416, 416))
        img /= 255.0
        
        if 'box_new' in ann.keys():
            ann = ann['box_new']
        elif 'anno_file' in ann.keys():
            ann = ann['anno_file']
        elif 'anno_previous' in ann.keys():
            ann = ann['anno_previous']
        else:
            raise ValueError("Invalid Annotation Error")

        # get bounding box coordinates for each mask.
        bbox = [a[1:] for a in ann]
        for i in bbox:
            xmin = i[0]
            xmax = xmin + i[2]
            ymin = i[1]
            ymax = ymin + i[3]
            xmin = (xmin/image_width)*416
            xmax = (xmax/image_width)*416
            ymin = (ymin/image_height)*416
            ymax = (ymax/image_height)*416
            i[:] = [xmin, ymin, xmax, ymax]
        
        # store boxes as a tensor.
        boxes = torch.as_tensor(bbox, dtype=torch.float32)
        # there is only one class.
        labels = torch.ones((len(bbox),), dtype=torch.int32)
        
        # area of the bounding boxes
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        # no crowd instances
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        
        # prepare the final `target` dictionary
        target = {}
        target["boxes"] = torch.FloatTensor(boxes)
        target["labels"] = labels
        target["area"] = area
        target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id

        # apply transform if required.
        if self.transforms:
            sample = self.transforms(image = img,
                                     bboxes = target['boxes'],
                                     labels = labels)
            for b in sample['boxes']:
                (x1, y1, x2, y2) = b
                for a in range(4):
                    b[a] = math.floor(b[a]) if b[a] > 1 else b[a]
                    b[a] = math.ceil(b[a]) if b[a] < 0 else b[a]
                        
            image_resized = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            
        return image_resized, target

    def __len__(self):
        """Gets the size of the dataset (number of samples)."""
        
        return len(self.imgs)

Defines the transforms for our image. Most of the preprocessing is built into the existing model, so we just need to convert to tensors.

In [4]:
# define the training tranforms
def get_transform():
    return A.Compose([ToTensorV2(p=1.0),], 
                     bbox_params={'format': 'pascal_voc',
                                  'label_fields': ['labels']})

Dataloader helper. The dataset will output variable size data so we need to work with a custom data getter.

In [5]:
def collate_fn(batch):
    """Dataloader data gatherer."""
    
    return tuple(zip(*batch))

Create a Torch device for memory management and GPU processes and create an instance of the model.

In [6]:
# set the device.
device = device('mps' if torch.has_mps else 'cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)
print('Model Set')

Model Set


Create a training function.

In [8]:
# function for running training iterations
def train(train_data_loader, model):
    """Trains the model."""
    
    # initialize the total training and validation loss
    totalTrainLoss = 0
    
    for data in train_data_loader:
        # zero oput the gradients.
        optimizer.zero_grad()
        images, targets = data
        
        # move the target data and the image to device.
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # calculate the loss.
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        # record the loss. (Tracking)
        totalTrainLoss += loss_value
        
        # backpropagate the error.
        losses.backward()
        optimizer.step()
        
    return totalTrainLoss

Create a validation function.

In [9]:
# function for running validation iterations
def validate(valid_data_loader, model):
    """Validates the model."""
    
    totalValLoss = 0
    
    # initialize tqdm progress bar
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
    
    for i, data in enumerate(prog_bar):
        # unpack the data.
        images, targets = data
        
        # move images and target data to device. 
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # turn off gradient tracking.
        with torch.no_grad():
            loss_dict = model(images, targets)
            
        # calculate the loss.
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        totalValLoss += loss_value
        
    return totalValLoss

Now lets generate the dataset and load it to a Torch dataloader for training and evaluation.

In [10]:
BATCH = 5

# define required data directories.
img_path = r'/Users/chiahaohsutai/Documents/GitHub/PRW/images/frames'
ann_path = r'/Users/chiahaohsutai/Documents/GitHub/PRW/annotations'

# use our dataset and defined transformations
dataset = prw.PRWTransfer(img_path, ann_path, get_transform())
dataset_test = prw.PRWTransfer(img_path, ann_path, get_transform())

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-1000])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-1000:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=BATCH, shuffle=True, num_workers=4,
    collate_fn=prw.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=BATCH, shuffle=False, num_workers=4,
    collate_fn=prw.collate_fn)

Train the model.

In [ ]:
# freeze the backbone of the model.
for p in model.backbone.parameters():
    p.requires_grad = False

NUM_EPOCHS = 1
trainSteps = len(dataset) // BATCH
valSteps = len(dataset_test) // BATCH

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# for tracking histories.
train_loss_list = []
val_loss_list = []

# start the training epochs
for epoch in range(NUM_EPOCHS):
    
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
    
    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(data_loader, model)
    val_loss = validate(data_loader_test, model)
    
    lr_scheduler.step()
    
    # print metrics.
    print(f"Epoch #{epoch+1} train loss: {train_loss:.3f}")   
    print(f"Epoch #{epoch+1} validation loss: {val_loss:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
    
    # calculate the average training and validation loss
    avgTrainLoss = train_loss / trainSteps
    avgValLoss = val_loss / valSteps
    
    train_loss_list.append(avgTrainLoss)
    val_loss_list.append(avgValLoss)


EPOCH 1 of 1


[W MPSFallback.mm:11] Warning: The operator 'torchvision::nms' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (function operator())


[(86.55974006652832, 156.97877883911133, 98.68932390213013, 203.70014476776123), (56.74115991592407, 121.9377326965332, 76.45174217224121, 258.5079288482666), (29.954983472824097, 102.17099475860596, 53.20336437225342, 257.6094379425049), (113.85131454467773, 147.09540367126465, 124.97010231018066, 208.1925868988037), (212.17869186401367, 159.28139114379883, 221.5924892425537, 209.9242820739746), (219.4798355102539, 165.96365070343018, 230.0932159423828, 203.70014476776123)]
[(273.5575771331787, 156.97877883911133, 281.13858795166016, 189.32433986663818), (0.6418106555938721, 112.05436992645264, 26.417187929153442, 245.92908096313477), (28.438786268234253, 112.95284843444824, 56.74115991592407, 237.84268760681152), (184.60725021362305, 161.4712085723877, 192.18824863433838, 193.81678199768066)]
[(299.8772659301758, 123.22670364379883, 311.7548713684082, 175.51340198516846), (282.6725387573242, 125.30053997039795, 294.33777046203613, 189.61712551116943), (138.30398654937744, 134.3375196

[(376.28797149658203, 107.78860855102539, 388.848669052124, 177.9560718536377), (399.02334213256836, 109.35494232177734, 412.306978225708, 181.1686429977417), (93.12993001937866, 141.70447063446045, 130.0240888595581, 304.3308162689209)]
[(386.39403533935547, 110.44092464447021, 396.9467658996582, 153.88610649108887), (56.48216438293457, 137.59416484832764, 79.8092679977417, 268.91711807250977)]
[(91.61373281478882, 141.70447063446045, 126.48629951477051, 295.34595680236816)]
[(282.7019462585449, 165.57734870910645, 294.4374485015869, 233.24083518981934), (191.00441074371338, 166.3405303955078, 204.83462715148926, 246.16401863098145), (372.6159019470215, 137.21204090118408, 406.9830551147461, 338.4733467102051), (226.5554141998291, 177.64399528503418, 248.79298973083496, 300.73687744140625), (206.84483814239502, 169.55760192871094, 224.02842712402344, 245.92908096313477)]
[(145.90272617340088, 149.93655014038086, 159.23249435424805, 198.31868362426758), (23.384790420532227, 147.9938821

[(409.60833168029785, 97.15877628326416, 415.8257122039795, 146.39548778533936), (135.4971570968628, 132.57003211975098, 144.2840118408203, 179.3605899810791), (331.04311752319336, 125.29404354095459, 342.3474750518799, 183.5742311477661), (77.26050424575806, 158.4534559249878, 137.73877239227295, 414.68534088134766)]
[(175.9653377532959, 161.35175609588623, 187.20182991027832, 223.35924530029297), (196.19697952270508, 162.1316499710083, 207.96748065948486, 224.62622261047363), (61.79515886306763, 152.48632431030273, 103.74332284927368, 350.1537284851074)]
[(147.1642484664917, 154.70066452026367, 168.53619956970215, 280.64887619018555), (259.9118022918701, 157.87725734710693, 270.525182723999, 203.70014476776123), (225.38094902038574, 161.60054206848145, 243.73199844360352, 254.92807388305664), (397.380464553833, 162.3696994781494, 410.52088165283203, 215.38048934936523), (30.460384607315063, 123.73471450805664, 54.71956157684326, 231.55327606201172), (58.76275825500488, 121.9377326965

...

[(377.9366264343262, 158.77573585510254, 390.5716361999512, 196.51225471496582), (49.665568828582764, 121.9377326965332, 65.83835554122925, 226.16235542297363), (276.15500831604004, 163.48589515686035, 288.32301902770996, 235.69467544555664), (283.12540435791016, 161.99529933929443, 298.3421001434326, 237.9479694366455)]
[(357.95931243896484, 103.06947946548462, 414.56406593322754, 364.52950859069824), (139.6266918182373, 136.31353759765625, 163.38047695159912, 251.32002639770508), (26.92258906364441, 121.03924179077148, 50.67636799812317, 219.87294387817383)]
[(175.67721366882324, 156.68133163452148, 191.14438152313232, 236.26492309570312), (324.98398208618164, 165.77700233459473, 338.9298324584961, 240.42305946350098), (357.2058506011963, 168.79559803009033, 370.0611686706543, 240.83984756469727), (49.1601676940918, 137.21204090118408, 64.82755327224731, 224.3653736114502), (68.36535501480103, 131.82110786437988, 86.55974006652832, 222.56841659545898), (145.69148063659668, 154.283306

[(160.34807014465332, 205.49712657928467, 170.45606803894043, 236.0457305908203), (274.56839179992676, 224.3653736114502, 335.21632957458496, 415.74334144592285), (195.72603797912598, 220.77143478393555, 208.86643028259277, 277.37618827819824), (358.46469497680664, 243.23363304138184, 397.88587188720703, 413.0478935241699)]
[(168.43446350097656, 160.57271766662598, 177.53165912628174, 201.0046844482422), (180.05865859985352, 161.4712085723877, 189.6612491607666, 199.20771503448486), (318.53813552856445, 165.06517219543457, 340.2703285217285, 254.91396522521973), (279.6223659515381, 158.77573585510254, 297.8167324066162, 254.91396522521973), (126.26367282867432, 167.32377243041992, 135.1871633529663, 212.56589889526367), (119.38516902923584, 158.76632595062256, 127.80967426300049, 202.7204236984253)]
[(169.9561653137207, 132.32786655426025, 182.3909511566162, 181.56457805633545), (129.5187063217163, 136.31353759765625, 143.16449356079102, 182.13644981384277)]
[(122.44310283660889, 184.8

[(75.17436933517456, 132.98941135406494, 92.0851640701294, 221.21001815795898), (114.86211681365967, 131.82110786437988, 139.1212968826294, 237.84268760681152)]
[(78.39094495773315, 144.3858346939087, 90.26050424575806, 186.58873558044434), (186.34746265411377, 135.34234619140625, 219.13006973266602, 253.9123992919922), (113.43441486358643, 115.24574089050293, 142.8257246017456, 260.94622230529785)]
[(341.7822608947754, 105.19742584228516, 354.2170467376709, 193.6225461959839), (344.60833168029785, 102.18292760848999, 363.260498046875, 195.63219928741455), (28.438786268234253, 147.99388217926025, 63.81675720214844, 287.2595634460449)]
[(356.3462886810303, 59.60161590576172, 377.40957260131836, 157.89696979522705), (275.49579429626465, 71.79091358184814, 307.122594833374, 192.18832302093506)]
[(169.4481544494629, 41.58512020111084, 198.24386310577393, 141.06736183166504), (279.0897579193115, 63.70452642440796, 297.0595016479492, 133.78658771514893)]
[(102.30221271514893, 243.73276710510

[(76.16787052154541, 220.38088035583496, 94.67401599884033, 326.3259906768799), (326.1191463470459, 239.63966941833496, 345.32430267333984, 349.2552127838135), (193.1990509033203, 221.66992568969727, 207.85564041137695, 300.73687744140625), (346.3351173400879, 216.27898025512695, 363.5187187194824, 302.53383445739746), (391.31567573547363, 236.94422149658203, 409.510066986084, 307.0262641906738)]
[(98.18392896652222, 136.31353759765625, 112.8405122756958, 201.0046844482422), (152.2616891860962, 130.02412605285645, 164.89666175842285, 184.8318977355957), (317.52734565734863, 196.51225471496582, 398.3912544250488, 411.2509117126465)]
[(169.95067310333252, 223.46688270568848, 185.61805248260498, 309.721736907959), (241.71741104125977, 209.98956871032715, 252.33079147338867, 257.6094379425049), (353.9161033630371, 231.55327606201172, 372.11049461364746, 342.06731033325195), (60.148332595825195, 217.70526885986328, 69.21827173233032, 264.0764350891113)]
[(338.38341331481934, 51.888382911682

[(90.91741228103638, 141.05002975463867, 103.13636541366577, 200.2934741973877), (99.80392646789551, 140.0626344680786, 113.13369464874268, 202.2682523727417), (17.048253417015076, 149.93655014038086, 33.71046829223633, 227.94039344787598)]
[(90.09753561019897, 157.87725734710693, 99.70012617111206, 204.59863567352295), (392.32646560668945, 156.0802755355835, 416.0, 263.8988742828369), (139.6266918182373, 153.38481521606445, 160.85346508026123, 262.10186767578125)]
[(299.8383369445801, 160.57271766662598, 308.93554496765137, 207.294095993042), (239.19039916992188, 162.3696994781494, 250.81459426879883, 227.06084632873535), (102.22712564468384, 147.99388217926025, 114.35670948028564, 212.68501663208008), (119.91610336303711, 156.97877883911133, 132.0457057952881, 217.17747116088867)]
[(153.272479057312, 127.32865333557129, 186.6288423538208, 239.63966941833496)]
[(255.86859321594238, 165.06517219543457, 278.61157608032227, 241.4366512298584), (78.12115716934204, 179.0796194076538, 87.16

In [ ]:
# path to current folder.
curr_dir = r'/Users/chiahaohsutai/Documents/GitHub/PedestrianDetection/save'
model_dir = os.path.join(curr_dir, 'model_transfer.pth')

# serialize the model to disk
print("[INFO] saving object detector model...")
torch.save(model, model_dir)

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(12, 8))
plt.plot(train_loss_list, label="total_train_loss")
plt.plot(val_loss_list, label="total_val_loss")
plt.title("Total Training and Validation Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

# save the training plot
plotPath = os.path.sep.join([curr_dir, "training_transfer.png"])
plt.savefig(plotPath)

# show the plot.
plt.show()

In [ ]:
# move the model to device.
model.eval().to(device)
print('Model in Eval Mode')

In [ ]:
def predict(photo, model, device, thresh):
    
    with torch.no_grad():
        outputs = model(photo.to(DEVICE))
    
    # load all detection to CPU for further operations
    outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]

    # carry further only if there are detected boxes
    if len(outputs[0]['boxes']) != 0:
        # get the boxes and the scores.
        boxes = outputs[0]['boxes'].data.numpy()
        scores = outputs[0]['scores'].data.numpy()
        labels = outputs[0]['labels'].cpu().numpy()

        # filter out boxes according to `detection_threshold`
        labels = labels[scores >= thresh].astype(np.int32)
        boxes = boxes[scores >= thresh].astype(np.int32)
        boxes = boxes[labels == 1]

    return boxes

In [ ]:
def iou(boxA, boxB):
    """Calculates Intersection Over Union."""
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # return the intersection over union value
    return iou

def get_boxes(ann):
    """Gets the correct key for annotations."""
    
    keys = ann.keys()
    key = None
    
    # get the correct key for the bouding box.
    for k in ['box_new', 'anno_file', 'anno_previous']:
        if k in keys:
            key = k
            break
    if key is None:
        raise ValueError("Invalid Annotation Error")
    
    # get the bounding boxes and convert to coordinates.
    bbox = [box[1:] for box in ann[key]]
    for box in bbox:
        xmin, ymin, w, h = box
        xmax, ymax = xmin+w, ymin+h
        box[-2] = xmax
        box[-1] = ymax
        
    return bbox

In [ ]:
# get the image names.
images_names = sorted(list(listdir(img_path)))
img_names = [join(img_path, name) for name in img_names]

# get the annoation names.
ann_names = sorted(list(listdir(ann_path)))
ann_names = [join(ann_path, name) for name in ann_names]

In [ ]:
total_iou = []
# truth, pred
total_count = []
total = len(img_names)

# evaluate the model.
for idx in indices[-1000:]
    img, ann = img_names[idx], ann_names[idx]
    
    # get appropiate data.
    data = loadmat(ann)
    photo = cv2.imread(test_images[i])
    photo = cv2.cvtColor(photo, cv2.COLOR_BGR2RGB).astype(np.float32)
    # make the pixel range between 0 and 1
    photo /= 255.0
    
    # bring color channels to front
    photo = np.transpose(image, (2, 0, 1)).astype(np.float32)
    # convert to tensor
    photo = torch.tensor(image, dtype=torch.float).cuda()
    # add batch dimension
    photo = torch.unsqueeze(image, 0)
    
    gt = get_boxes(data)
    
    # get a prediction
    with torch.no_grad():
        pred = predict(photo, model, device, 0.7)
    
    # go through the prediciton and calulate iou
    boxes, labels, _ = pred
    for bbox in boxes:
        # get the iou.
        union = 0
        for b in gt:
            union = max(union, iou(b, bbox))
        if union > 0.25:
            total_iou.append(union)
    
    # get a count of the boxes.
    total_count.append((len(gt), len(labels)))
    
    if len(total_count) == 500:
        break

In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [ ]:
# extract the counts.
y_true = [c[0] for c in total_count]
y_pred = [c[1] for c in total_count]

mse_eval = MSE(y_true, y_pred, squared=False)
mape_eval = MAPE(y_true, y_pred)
avg_iou = sum(total_iou) / len(total_iou)

In [ ]:
print(f'RMSE: {mse_eval}, MAPE: {mape_eval}, Avg. IOU: {avg_iou}')